# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1622489410000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2021-04-30&endtime=2021-05-30',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.10.3',
 'count': 10232}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 2,
  'place': '23 km N of Willow, Alaska',
  'time': 1622332708509,
  'updated': 1622333534854,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak0216ut468m',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak0216ut468m&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 62,
  'net': 'ak',
  'code': '0216ut468m',
  'ids': ',ak0216ut468m,',
  'sources': ',ak,',
  'types': ',origin,',
  'nst': None,
  'dmin': None,
  'rms': 0.62,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 2.0 - 23 km N of Willow, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-150.0831, 61.9551, 30.1]},
 'id': 'ak0216ut468m'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,2.00,"23 km N of Willow, Alaska",1622332708509,1622333534854,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak0216ut468m,",",ak,",",origin,",NaN,NaN,0.62,NaN,ml,earthquake,"M 2.0 - 23 km N of Willow, Alaska"
1,0.35,"2km N of The Geysers, CA",1622331217450,1622338572406,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73567366,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",14.0,0.005785,0.02,120.0,md,earthquake,"M 0.4 - 2km N of The Geysers, CA"
2,4.50,"91 km W of San Antonio de los Cobres, Argentina",1622330654734,1622398852359,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.0,1.0,NaN,...,",us6000efw3,",",us,",",dyfi,origin,phase-data,",NaN,1.451000,0.51,106.0,mb,earthquake,"M 4.5 - 91 km W of San Antonio de los Cobres, ..."
3,2.16,"18km E of Ocotillo, CA",1622330638080,1622330856366,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39909792,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",25.0,0.092780,0.28,57.0,ml,earthquake,"M 2.2 - 18km E of Ocotillo, CA"
4,4.40,"115 km WNW of San Antonio de los Cobres, Argen...",1622330295993,1622332202040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us6000efw1,",",us,",",origin,phase-data,",NaN,1.111000,0.44,100.0,mb,earthquake,M 4.4 - 115 km WNW of San Antonio de los Cobre...


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>